# **Setup and Imports**

In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import classification_report, confusion_matrix
sns.set(style="whitegrid", palette="colorblind", font_scale=1.2)

ModuleNotFoundError: No module named 'tensorflow'

# **Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/intel_image_classification/"

# **Load and Explore the Dataset**

In [ ]:
import os
#Train Path
train_path = "/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train"

def explore_dataset(train_dir, output_path="class_distribution.png"):
    class_counts = {}
    class_labels = []

    for class_name in sorted(os.listdir(train_dir)):
        class_path = os.path.join(train_dir, class_name)
        if os.path.isdir(class_path) and not class_name.startswith("."):
            image_files = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
            count = len(image_files)
            if count > 0:
                class_counts[class_name] = count
                class_labels.append(class_name)

    # Plot class distribution
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
    plt.title("Number of Training Images per Class")
    plt.xlabel("Class")
    plt.ylabel("Image Count")
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Save the plot
    plt.savefig(output_path)
    print(f"📁 Class distribution plot saved as: {output_path}")
    plt.show()

    return class_counts

# 🔁 Run the function
class_counts = explore_dataset(train_path)


# **Show Sample Images per Class**

In [ ]:
import matplotlib.image as mpimg
def show_sample_images(train_dir, output_path="sample_images.png"):
    class_names = sorted(os.listdir(train_dir))
    class_names = [cls for cls in class_names if not cls.startswith(".")]

    plt.figure(figsize=(18, 6))

    for idx, cls in enumerate(class_names):
        img_folder = os.path.join(train_dir, cls)
        img_file = random.choice([
            f for f in os.listdir(img_folder)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        img_path = os.path.join(img_folder, img_file)
        img = mpimg.imread(img_path)

        plt.subplot(1, len(class_names), idx + 1)
        plt.imshow(img)
        plt.title(cls, fontsize=12)
        plt.axis('off')

    plt.suptitle("Random Sample Image from Each Class", fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"Sample image grid saved as: {output_path}")
    plt.show()

# Use the correct path
show_sample_images("/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train")


# **Rotation Augmentation**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
import random

def preview_rotation_augmentation(data_dir, target_size=(224, 224), num_images=6, output_path="aug_rotation.png"):
    datagen = ImageDataGenerator(rotation_range=40)

    # Get class folders
    class_folders = [d for d in os.listdir(data_dir) if not d.startswith(".")]
    sample_images = []

    # Pick random images from random classes
    for _ in range(num_images):
        class_choice = random.choice(class_folders)
        img_folder = os.path.join(data_dir, class_choice)
        img_file = random.choice([f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        img_path = os.path.join(img_folder, img_file)
        sample_images.append((img_path, class_choice))

    # Plot
    plt.figure(figsize=(18, 8))
    for i, (img_path, label) in enumerate(sample_images):
        # Load and preprocess
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        # Generate rotated image
        aug_iter = datagen.flow(img_array, batch_size=1)
        rotated_img = next(aug_iter)[0].astype("uint8")

        # Plot original
        plt.subplot(2, num_images, i + 1)
        plt.imshow(img_array[0].astype("uint8"))
        plt.title(f"Original\n{label}")
        plt.axis('off')

        # Plot rotated
        plt.subplot(2, num_images, num_images + i + 1)
        plt.imshow(rotated_img)
        plt.title(f"Rotated\n{label}")
        plt.axis('off')

    plt.suptitle("Rotation Augmentation Preview", fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"✅ Rotation augmentation saved as: {output_path}")
    plt.show()

# 🔁 Run it
preview_rotation_augmentation("/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train")


# **CLAHE Contrast Enhancement**

In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
import os

def apply_clahe(image_path, tile_grid_size=(8, 8), clip_limit=2.0):
    img = cv2.imread(image_path)  # BGR format
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l)

    limg = cv2.merge((cl, a, b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)  # convert to RGB for display
    original_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return original_rgb, final

def preview_clahe_augmentation(data_dir, num_images=6, output_path="aug_clahe.png"):
    class_folders = [d for d in os.listdir(data_dir) if not d.startswith(".")]
    sample_images = []

    for _ in range(num_images):
        class_choice = random.choice(class_folders)
        img_folder = os.path.join(data_dir, class_choice)
        img_file = random.choice([f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        img_path = os.path.join(img_folder, img_file)
        sample_images.append((img_path, class_choice))

    # Plot
    plt.figure(figsize=(18, 8))
    for i, (img_path, label) in enumerate(sample_images):
        orig, enhanced = apply_clahe(img_path)

        plt.subplot(2, num_images, i + 1)
        plt.imshow(orig)
        plt.title(f"Original\n{label}")
        plt.axis('off')

        plt.subplot(2, num_images, num_images + i + 1)
        plt.imshow(enhanced)
        plt.title(f"CLAHE\n{label}")
        plt.axis('off')

    plt.suptitle("CLAHE Contrast Enhancement Preview", fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"✅ CLAHE augmentation saved as: {output_path}")
    plt.show()

# 🔁 Run it
preview_clahe_augmentation("/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train")


# **Horizontal Flip Augmentation**

In [ ]:
def preview_horizontal_flip(data_dir, target_size=(224, 224), num_images=6, output_path="aug_flip.png"):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
    import matplotlib.pyplot as plt
    import numpy as np
    import os
    import random

    datagen = ImageDataGenerator(horizontal_flip=True)

    # Force use of 'street' class for clear visual flips
    class_choice = "street"
    img_folder = os.path.join(data_dir, class_choice)
    all_files = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Pick random images only from 'street'
    sample_images = random.sample(all_files, num_images)

    plt.figure(figsize=(18, 8))
    for i, img_file in enumerate(sample_images):
        img_path = os.path.join(img_folder, img_file)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array_expanded = np.expand_dims(img_array, axis=0)

        flipped_img = next(datagen.flow(img_array_expanded, batch_size=1))[0].astype("uint8")

        # Original
        plt.subplot(2, num_images, i + 1)
        plt.imshow(img_array.astype("uint8"))
        plt.title(f"Original\n{class_choice}")
        plt.axis('off')

        # Flipped
        plt.subplot(2, num_images, num_images + i + 1)
        plt.imshow(flipped_img)
        plt.title(f"Flipped\n{class_choice}")
        plt.axis('off')

    plt.suptitle("Horizontal Flip Augmentation Preview (Street Images)", fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"✅ Horizontal flip augmentation saved as: {output_path}")
    plt.show()

# Run it!
preview_horizontal_flip("/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train")


# **Zoom & Crop Augmentation Preview**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np
import random
import os

def preview_zoom_augmentation(data_dir, target_size=(224, 224), num_images=6, output_path="aug_zoom.png"):
    datagen = ImageDataGenerator(zoom_range=0.3)  # Zoom in 0–30%

    # Choose a visually rich class (e.g., 'mountain' or 'forest')
    class_choice = "mountain"
    img_folder = os.path.join(data_dir, class_choice)
    all_files = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    sample_images = random.sample(all_files, num_images)

    plt.figure(figsize=(18, 8))
    for i, img_file in enumerate(sample_images):
        img_path = os.path.join(img_folder, img_file)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array_expanded = np.expand_dims(img_array, axis=0)

        zoomed_img = next(datagen.flow(img_array_expanded, batch_size=1))[0].astype("uint8")

        # Original
        plt.subplot(2, num_images, i + 1)
        plt.imshow(img_array.astype("uint8"))
        plt.title(f"Original\n{class_choice}")
        plt.axis('off')

        # Zoomed
        plt.subplot(2, num_images, num_images + i + 1)
        plt.imshow(zoomed_img)
        plt.title(f"Zoomed\n{class_choice}")
        plt.axis('off')

    plt.suptitle("Zoom Augmentation Preview", fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"✅ Zoom augmentation saved as: {output_path}")
    plt.show()

preview_zoom_augmentation("/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train")


# **Gaussian Blur**

In [ ]:
def apply_blur(image_path, ksize=(5, 5)):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    blurred = cv2.GaussianBlur(img_rgb, ksize, 0)
    return img_rgb, blurred

def preview_blur_augmentation(data_dir, num_images=6, output_path="aug_blur.png"):
    class_choice = "glacier"  # visually sharp class works well
    img_folder = os.path.join(data_dir, class_choice)
    all_files = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    sample_images = random.sample(all_files, num_images)

    plt.figure(figsize=(18, 8))
    for i, img_file in enumerate(sample_images):
        img_path = os.path.join(img_folder, img_file)
        original, blurred = apply_blur(img_path)

        # Original
        plt.subplot(2, num_images, i + 1)
        plt.imshow(original)
        plt.title(f"Original\n{class_choice}")
        plt.axis('off')

        # Blurred
        plt.subplot(2, num_images, num_images + i + 1)
        plt.imshow(blurred)
        plt.title(f"Blurred\n{class_choice}")
        plt.axis('off')

    plt.suptitle("Gaussian Blur Augmentation Preview", fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"✅ Blur augmentation saved as: {output_path}")
    plt.show()

# Run it
preview_blur_augmentation("/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train")


# **Data Preprocessing Pipeline for Model Training**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = "/content/drive/MyDrive/Colab Notebooks/intel_image_classification/seg_train/seg_train"

# Image size & batch settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Augmentations for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% train, 20% validation
)

#  No augmentation for validation, only rescaling
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Create the generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


# **Visualize Augmented Images from Generator**

In [ ]:
import matplotlib.pyplot as plt

def preview_augmented_batch(generator, class_names, output_path="batch_preview.png"):
    images, labels = next(generator)  # Get one batch

    plt.figure(figsize=(16, 8))
    for i in range(8):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        label_index = labels[i].argmax()
        plt.title(class_names[label_index])
        plt.axis('off')

    plt.suptitle("Sample Augmented Images from Training Generator", fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f" Augmented batch preview saved as: {output_path}")
    plt.show()

# Class labels from generator
class_names = list(train_generator.class_indices.keys())

# 🔁 Run it
preview_augmented_batch(train_generator, class_names)


# **Model Implementation**

**ResNet50 Transfer Learning**

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

def build_resnet_model(input_shape=(224, 224, 3), num_classes=6, learning_rate=0.0001):
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
    base_model.trainable = False  # Freeze base layers

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("✅ ResNet50 model built and compiled.")
    return model

# Build the model
resnet_model = build_resnet_model()
resnet_model.summary()


**Train RestNet50**

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Callbacks
checkpoint_path = "resnet50_best_model.h5"

checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# 📊 Train the model
history = resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[checkpoint, early_stop],
    verbose=1
)